# Subspace Model

This notebook illustrate how to use a Bayesian Normal density model with the [beer framework](https://github.com/beer-asr/beer). The Normal distribution is a fairly basic models but it is used extenslively in other model as a basic building block.

In [1]:
# Add "beer" to the PYTHONPATH
import sys
sys.path.insert(0, '../')

import beer
import numpy as np
import torch

# For plotting.
from bokeh.io import show, output_notebook
from bokeh.plotting import figure, gridplot
from bokeh.models import Arrow, OpenHead, NormalHead, VeeHead
output_notebook()

# Convenience functions for plotting.
import plotting

%load_ext autoreload
%autoreload 2

Loading BokehJS ...

## Data

Generate some normally distributed data:

In [2]:
mean = np.array([-10, 10]) * 0
cov = np.array([[1, -.75], [-.75, 1]])
#cov = np.eye(2) 
data = np.random.multivariate_normal(mean, cov, size=500)

x_range = (mean[0] - 5, mean[0] + 5)
y_range = (mean[1] - 5, mean[1] + 5)

fig = figure(
    title='Data',
    width=400,
    height=400,
    x_range=x_range,
    y_range=y_range
)
fig.circle(data[:, 0], data[:, 1])
plotting.plot_normal(fig, mean, cov, alpha=.5, color='black')

show(fig)

## Model Creation

We create two types of Normal distribution: one diagonal covariance matrix and another one with full covariance matrix.

In [18]:
model = beer.PPCA.create(torch.zeros(2).double(), 1, torch.eye(2).double())
#model = beer.NormalDiagonalCovariance.create(torch.zeros(2).double(), torch.ones(2).double())

## Variational Bayes Training 

In [19]:
epochs = 100
lrate = 1
X = torch.from_numpy(data).double()
elbo_fn = beer.EvidenceLowerBound(len(X))
params = [model.precision_param]
#params = model.parameters
optimizer = beer.BayesianModelOptimizer(params, lrate)
    
elbos = []
klds = []
for epoch in range(epochs):
    optimizer.zero_grad()
    elbo = elbo_fn(model, X)
    elbo.natural_backward()
    #print(float(elbo._kl_div), float(elbo._exp_llh_per_frame.sum()))
    optimizer.step()
    elbos.append(round(float(elbo) / len(X), 5))
fig = figure(width=400, height=400)
fig.line(range(len(elbos)), elbos)
show(fig)

In [5]:
model.precision, model.mean, model.subspace

(tensor(1., dtype=torch.float64),
 tensor([ 0.,  0.], dtype=torch.float64),
 tensor([[ 0.4241, -0.4159],
         [-0.4159,  0.4129]], dtype=torch.float64))

In [6]:
mean = model.mean.numpy()
evectors = model.subspace.numpy().T
print('bases:', evectors, evectors.shape)

fig1 = figure(
    title='PPCA',
    x_range=x_range,
    y_range=y_range,
    width=400,
    height=400
)

fig1.circle(data[:, 0], data[:, 1], alpha=.3)

# Plot subspace.
alpha = np.linalg.norm(evectors[:, 0])
alpha = 1
fig1.add_layout(
    Arrow(
        end=NormalHead(size=10, line_alpha=alpha, fill_alpha=alpha),
        line_width=2,
        line_alpha=alpha,
        x_start=mean[0], 
        y_start=mean[1], 
        x_end=evectors[0, 0] + mean[0], 
        y_end=evectors[1, 0] + mean[1])
)
alpha = np.linalg.norm(evectors[:, 1])
alpha = 1
fig1.add_layout(
    Arrow(
        end=NormalHead(size=10, line_alpha=alpha, fill_alpha=alpha),
        line_width=2,
        line_alpha=alpha,
        x_start=mean[0], 
        y_start=mean[1], 
        x_end=evectors[0, 1] + mean[0], 
        y_end=evectors[1, 1] + mean[1])
)

X = data
C = cov
evals, evectors = np.linalg.eigh(C)
print('evectors')
print(evectors)

fig2 = figure(
    title='PCA',
    x_range=x_range,
    y_range=y_range,
    width=400,
    height=400
)

fig2.circle(data[0], data[1], alpha=.3)

# Plot subspace.
alpha = np.linalg.norm(evectors[:, 1])
fig2.add_layout(
    Arrow(
        end=NormalHead(size=10, line_alpha=alpha, fill_alpha=alpha),
        line_width=2,
        line_alpha=alpha,
        x_start=mean[0], 
        y_start=mean[1], 
        x_end=evectors[0, 0] + mean[0], 
        y_end=evectors[1, 0] + mean[1])
)
alpha = np.linalg.norm(evectors[:, 1])
fig2.add_layout(
    Arrow(
        end=NormalHead(size=10, line_alpha=alpha, fill_alpha=alpha),
        line_width=2,
        line_alpha=alpha,
        x_start=mean[0], y_start=mean[1], 
        x_end=evectors[0, 1] + mean[0], 
        y_end=evectors[1, 1] + mean[1])
)

grid = gridplot([[fig1, fig2]])
show(grid)

bases: [[ 0.42414774 -0.41587612]
 [-0.41587612  0.41291204]] (2, 2)
evectors
[[-0.70710678 -0.70710678]
 [-0.70710678  0.70710678]]


In [7]:
torch.lgamma

<function _VariableFunctions.lgamma>

In [8]:
from scipy.special import gammaln as lgamma
np.isclose(lgamma(10000000), torch.lgamma(torch.tensor(10000000.)).numpy())

True

In [9]:
_, _, _, _, m_quad, m_mean = model._get_expectation()
m_quad, m_mean = m_quad.numpy(), m_mean.numpy()

In [10]:
m_quad, m_quad - np.sum(m_mean**2)

(array(2.), 2.0)

In [11]:
m_mean, m_quad

(array([0., 0.]), array(2.))

In [12]:
model.mean_param.expected_value()

tensor([ 2.,  0.,  0.], dtype=torch.float64)